In [2]:
import pandas as pd

In [32]:
adv_stats = pd.read_csv("adv_stats.csv")
hist_odds = pd.read_csv("hist_odds.csv")
mom_stats = pd.read_csv("momentum.csv")
today_lines = pd.read_csv("today_lines.csv")
team_id = {'Washington' : 1610612764, 'GoldenState' : 1610612744, 'Minnesota' : 1610612750, 'Houston' : 1610612745, 'Milwaukee' : 1610612749, 'Chicago' : 1610612741, 'Philadelphia': 1610612755, 'Memphis' : 1610612763, 'Brooklyn' : 1610612751, 
           'OklahomaCity' : 1610612760, 'Sacramento' : 1610612758, 'Charlotte' : 1610612766, 'Indiana' : 1610612754, 'Toronto': 1610612761, 'SanAntonio' : 1610612759, 'Orlando' : 1610612753, 'Utah' : 1610612762, 'NewOrleans' : 1610612740, 'LALakers': 1610612747, 
           'Portland' : 1610612757, 'Dallas' : 1610612742, 'Atlanta' : 1610612737, 'Boston' : 1610612738, 'Cleveland' : 1610612739, 'Detroit': 1610612765, 'Miami' : 1610612748, 'LAClippers' : 1610612746, 'Denver' : 1610612743, 'Phoenix' : 1610612756, 'NewYork' : 1610612752}
hist_odds = hist_odds.replace(team_id)
hist_odds['win'] = [1 if x > y else 0 for x,y in zip(hist_odds['Final_H'], hist_odds['Final_V'])]
print(hist_odds)
hist_odds['fav'] = [1 if x > y else 0 for x,y in zip(hist_odds['ML_H'], hist_odds['ML_V'])]
home_team = hist_odds[["VH_y", "Team_H", "Final_H"]]
away_team = hist_odds[['VH_x', "Team_V", "Final_V"]]

home_team = pd.merge(left = home_team, right = adv_stats, left_on = "Team_H", right_on = "TEAM_ID", how = "left")
away_team = pd.merge(left = away_team, right = adv_stats, left_on = "Team_V", right_on = "TEAM_ID", how = "left")
model_stats = pd.merge(left = home_team, right = away_team, left_index = True, right_index = True, how = "outer")
model_stats['win'] = hist_odds['win']
model_stats['fav'] = hist_odds['fav']
model_stats.to_csv('model_stats.csv')

     Unnamed: 0  VH_x      Team_V  Final_V    O/U  ML_V  VH_y      Team_H  \
0             0     0  1610612744       99  228.0   265     1  1610612751   
1             1     0  1610612746      116  223.0   115     1  1610612747   
2             2     0  1610612766      114  215.0  -140     1  1610612739   
3             3     0  1610612752      107  214.5   270     1  1610612754   
4             4     0  1610612748      107  220.5  -190     1  1610612753   
..          ...   ...         ...      ...    ...   ...   ...         ...   
528         528     0  1610612760      107  218.5   230     1  1610612759   
529         529     0  1610612748      103  225.0  -130     1  1610612740   
530         530     0  1610612758      119  238.0   160     1  1610612757   
531         531     0  1610612765      104  211.0   245     1  1610612752   
532         532     0  1610612744       98  223.0   550     1  1610612756   

     Final_H   line  ML_H  win  
0        125    5.5  -320    1  
1        

Logistic Regression

In [37]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

to_keep = ['W_PCT_x', 'W_PCT_y', 'VH_y', 'E_OFF_RATING_x', 'E_OFF_RATING_y', 'E_DEF_RATING_x', 'E_DEF_RATING_y', 'E_NET_RATING_x', 'E_PACE_x', 'E_AST_RATIO_x', 'E_OREB_PCT_x' , 'E_DREB_PCT_x',
           'E_REB_PCT_x', 'E_TM_TOV_PCT_x', 'E_NET_RATING_y' , 'E_PACE_y', 'E_AST_RATIO_y', 'E_OREB_PCT_y' ,'E_DREB_PCT_y', 'E_REB_PCT_y', 'E_TM_TOV_PCT_y', 'MIN_x', 'MIN_y', 'fav']
model_stats_x = model_stats[to_keep]
col_names = ['W_PCT_HOME', 'W_PCT_AWAY', 'home_team', 'E_OFF_RATING_HOME', 'E_OFF_RATING_AWAY', 'E_DEF_RATING_HOME', 'E_DEF_RATING_AWAY', 'E_NET_RATING_HOME', 'E_PACE_HOME', 'E_AST_RATIO_HOME', 'E_OREB_PCT_HOME' , 'E_DREB_PCT_HOME',
           'E_REB_PCT_HOME', 'E_TM_TOV_PCT_HOME', 'E_NET_RATING_AWAY' , 'E_PACE_AWAY', 'E_AST_RATIO_AWAY', 'E_OREB_PCT_AWAY' ,'E_DREB_PCT_AWAY', 'E_REB_PCT_AWAY', 'E_TM_TOV_PCT_AWAY', 'MIN_HOME', 'MIN_AWAY', 'fav']
model_stats_x = model_stats[to_keep]
logit_model=sm.Logit(model_stats['win'],model_stats_x)
result=logit_model.fit()
print(result.summary())


X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['win'], test_size=0.3, random_state=0)
logreg = LogisticRegression(max_iter = 1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Optimization terminated successfully.
         Current function value: 0.612161
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    win   No. Observations:                  533
Model:                          Logit   Df Residuals:                      509
Method:                           MLE   Df Model:                           23
Date:                Fri, 12 Mar 2021   Pseudo R-squ.:                  0.1138
Time:                        20:43:09   Log-Likelihood:                -326.28
converged:                       True   LL-Null:                       -368.16
                                        LLR p-value:                 7.731e-09
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
W_PCT_x            5.0950      2.496      2.041      0.041       0.203       9.987
W_PCT_y          